In [12]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, KFold

Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [3]:
newsgroups = datasets.fetch_20newsgroups(
    subset='all', 
    categories=['alt.atheism', 'sci.space']
)

Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [42]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target

Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [44]:
param_grid = {
    'kernel': 'linear',
    'C': np.power(10, np.arange(-5, 6, dtype=float))
}

svc = SVC(random_state=241)
cv = KFold(n_splits=5, shuffle=True, random_state=241)

clf = GridSearchCV(svc, param_grid, scoring='accuracy', cv=cv)
clf.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [52]:
svc = SVC(kernel='linear', random_state=241, **clf.best_params_)
svc.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False)

Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [110]:
feature_mapping = np.array(vectorizer.get_feature_names())

In [106]:
for ind in np.argsort(abs(svc.coef_.toarray()[0]))[-10:]:
    print(ind, feature_mapping[ind])

22936 sci
15606 keith
5776 bible
21850 religion
23673 sky
17802 moon
5093 atheists
5088 atheism
12871 god
24019 space


In [116]:
answers = np.sort(feature_mapping[(np.argsort(abs(svc.coef_.toarray()[0]))[-10:])])
print(answers)
with open("tfidf_answer1.txt", "w") as fout:
    fout.write(" ".join([num.lower() for num in answers]))

['atheism' 'atheists' 'bible' 'god' 'keith' 'moon' 'religion' 'sci' 'sky'
 'space']
